# TDT4173 - Machine Learning
Names: Jacob Alexander Worsøe, Henrik August Søntvedt, Simen Øygarden Burgos

Student IDs: 544553, 543969, 543917

Team name: 5tabekk

Team number: 5



## Long Notebook 

We worked with a system where we created a python library containing helper functions. These have been placed below and adapted to the notebook format. This library also contained a MetaModel class that we created to more easily test and keep track of our models. This is why our models are written into classes.


### 1. Setup


#### Data location

We kept the data in a folder titled data as follows
 - data
    - A
    - B
    - C
    my_first_submission.csv
    read_files.ipynb
    Readme.md
    sample_submission.csv
    test.csv

In [ ]:
data_dir = './data/'


#### Imports

In [ ]:
import os

import pandas as pd
import numpy as np

from abc import ABC, abstractmethod
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn.metrics import mean_absolute_error
import statistics

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)

import sys
import random
import math
from autogluon.tabular import TabularDataset, TabularPredictor

from sklearn.linear_model import LinearRegression


from keras import models, layers, optimizers, regularizers
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf

import lightgbm as lgbm

from sklearn.model_selection import TimeSeriesSplit, train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.utils import timeseries_dataset_from_array
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf

from sklearn.decomposition import PCA


import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

from prophet import Prophet

import xgboost as xgb

from itertools import combinations


#### Data fetching and preprocessing functions

In [ ]:

def get_training():
    """ gets full training data (duh) """

    train_a = pd.read_parquet(data_dir + 'A/train_targets.parquet')
    train_b = pd.read_parquet(data_dir + 'B/train_targets.parquet')
    train_c = pd.read_parquet(data_dir + 'C/train_targets.parquet')

    X_train_estimated_a = pd.read_parquet(data_dir + 'A/X_train_estimated.parquet')
    X_train_estimated_b = pd.read_parquet(data_dir + 'B/X_train_estimated.parquet')
    X_train_estimated_c = pd.read_parquet(data_dir + 'C/X_train_estimated.parquet')

    X_train_observed_a = pd.read_parquet(data_dir + 'A/X_train_observed.parquet')
    X_train_observed_b = pd.read_parquet(data_dir + 'B/X_train_observed.parquet')
    X_train_observed_c = pd.read_parquet(data_dir + 'C/X_train_observed.parquet')

    ret = pd.DataFrame()
    ret.set_index(pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=['location', 'datetime']), inplace=True)
    ret.columns = pd.MultiIndex(levels=[[],[], []], codes=[[],[], []], names=['feature_type', 'minutes', 'feature_name'])


    # estimated data
    data_dict = {'A': X_train_estimated_a, 'B': X_train_estimated_b, 'C': X_train_estimated_c}

    for loc in data_dict:
        out_temp = pd.DataFrame(data_dict[loc].date_forecast.dt.floor('H').unique(), columns=['date_forecast'])
        out_temp.set_index('date_forecast', inplace=True)
        out_temp.columns = pd.MultiIndex.from_product([[], [], out_temp.columns], names=['feature_type', 'minutes', 'feature_name'])

        for m in [0, 15, 30, 45]:
            temp = data_dict[loc][data_dict[loc].date_forecast.dt.minute == m].copy()
            temp.set_index(temp.date_forecast.dt.floor('H'), inplace=True)
            temp.drop(columns=['date_calc', 'date_forecast'], inplace=True)
            temp.columns = pd.MultiIndex.from_product([['estimated'], [m], temp.columns], names=['feature_type', 'minutes', 'feature_name'])
            out_temp = out_temp.merge(temp,left_index=True, right_index=True, how='outer')

        out_temp.set_index(pd.MultiIndex.from_product([[loc], out_temp.index], names=['location', 'datetime']), inplace=True)
        ret = pd.concat([ret, out_temp])

    # observed data
    data_dict = {'A': X_train_observed_a, 'B': X_train_observed_b, 'C': X_train_observed_c}

    for loc in data_dict:
        out_temp = pd.DataFrame(data_dict[loc].date_forecast.dt.floor('H').unique(), columns=['date_forecast'])
        out_temp.set_index('date_forecast', inplace=True)
        out_temp.columns = pd.MultiIndex.from_product([[], [], out_temp.columns], names=['feature_type', 'minutes', 'feature_name'])

        for m in [0, 15, 30, 45]:
            temp = data_dict[loc][data_dict[loc].date_forecast.dt.minute == m].copy()
            temp.set_index(temp.date_forecast.dt.floor('H'), inplace=True)
            temp.drop(columns=['date_forecast'], inplace=True)
            temp.columns = pd.MultiIndex.from_product([['observed'], [m], temp.columns], names=['feature_type', 'minutes', 'feature_name'])
            out_temp = out_temp.merge(temp,left_index=True, right_index=True, how='outer')

        out_temp.set_index(pd.MultiIndex.from_product([[loc], out_temp.index], names=['location', 'datetime']), inplace=True)
        ret = pd.concat([ret, out_temp])
        


    # # train data
    data_dict = {'B': train_b, 'C': train_c} # 'A': train_a, 

    out_temp = train_a.dropna().copy()
    out_temp.rename(columns={'time': 'datetime'}, inplace=True)
    out_temp.set_index('datetime', inplace=True)
    out_temp.columns = pd.MultiIndex.from_product([['y'], ['NA'], out_temp.columns], names=['feature_type', 'minutes', 'feature_name'])

    out_temp.set_index(pd.MultiIndex.from_product([['A'], out_temp.index], names=['location', 'datetime']), inplace=True)

    for loc in data_dict:

        out_temp2 = data_dict[loc].dropna().copy()
        out_temp2.rename(columns={'time': 'datetime'}, inplace=True)
        out_temp2.set_index('datetime', inplace=True)
        out_temp2.columns = pd.MultiIndex.from_product([['y'], ['NA'], out_temp2.columns], names=['feature_type', 'minutes', 'feature_name'])

        out_temp2.set_index(pd.MultiIndex.from_product([[loc], out_temp2.index], names=['location', 'datetime']), inplace=True)

        out_temp = pd.concat([out_temp, out_temp2])

        # ret = ret.merge(out_temp, left_index=True, right_index=True, how='outer')
        # ret = pd.concat([ret, out_temp])
        

    ret = pd.merge(out_temp, ret, left_index=True, right_index=True, how='outer')

    return ret


def get_training_groupby_mean():
    return get_training().groupby(level = [0,2], axis = 1).mean()[['y', 'estimated', 'observed']]


def get_training_flattened():
    df = get_training_groupby_mean()

    df['estimated', 'weather_data_type'] = np.nan

    df['estimated', 'weather_data_type'] = np.where(df['observed', 'absolute_humidity_2m:gm3'].notna(),'observed', df['estimated', 'weather_data_type'])
    df['estimated', 'weather_data_type'] = np.where(df['estimated', 'absolute_humidity_2m:gm3'].notna(),'estimated', df['estimated', 'weather_data_type'])
    df.estimated = df.estimated.fillna(df.observed)

    df = df.drop(columns=['observed'], level=0)
    df.columns = df.columns.droplevel().tolist()
    df.reset_index(inplace=True)
    df.rename(columns={'datetime': 'ds', 'pv_measurement': 'y'}, inplace=True)
    # df['ENG_total_rad'] = df['diffuse_rad_1h:J'] + df['direct_rad_1h:J']
    
    return df[['location', 'ds', 'y', 'weather_data_type'] + [i for i in df.columns.tolist() if i not in ['location', 'ds', 'y', 'weather_data_type']]].copy()

def get_training_cleaned():
    df = get_training_flattened()

    df = df[
            ~((df.y != 0) & 
            (df.y == df.y.shift(-1)) &
            (df.y == df.y.shift(-2)))
        ].copy()

    df.dropna(axis=0, subset='y', inplace=True)

    return df


### Testing

def get_testing():
    """ gets the feature estimates used for the forecast """

    X_test_estimated_a = pd.read_parquet(data_dir + 'A/X_test_estimated.parquet')
    X_test_estimated_b = pd.read_parquet(data_dir + 'B/X_test_estimated.parquet')
    X_test_estimated_c = pd.read_parquet(data_dir + 'C/X_test_estimated.parquet')

    ret = pd.DataFrame()
    ret.set_index(pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=['location', 'datetime']), inplace=True)
    ret.columns = pd.MultiIndex(levels=[[],[], []], codes=[[],[], []], names=['feature_type', 'minutes', 'feature_name'])


    # estimated data
    data_dict = {'A': X_test_estimated_a, 'B': X_test_estimated_b, 'C': X_test_estimated_c}

    for loc in data_dict:
        out_temp = pd.DataFrame(data_dict[loc].date_forecast.dt.floor('H').unique(), columns=['date_forecast'])
        out_temp.set_index('date_forecast', inplace=True)
        out_temp.columns = pd.MultiIndex.from_product([[], [], out_temp.columns], names=['feature_type', 'minutes', 'feature_name'])

        for m in [0, 15, 30, 45]:
            temp = data_dict[loc][data_dict[loc].date_forecast.dt.minute == m].copy()
            temp.set_index(temp.date_forecast.dt.floor('H'), inplace=True)
            temp.drop(columns=['date_calc', 'date_forecast'], inplace=True)
            temp.columns = pd.MultiIndex.from_product([['estimated'], [m], temp.columns], names=['feature_type', 'minutes', 'feature_name'])
            out_temp = out_temp.merge(temp,left_index=True, right_index=True, how='outer')

        out_temp.set_index(pd.MultiIndex.from_product([[loc], out_temp.index], names=['location', 'datetime']), inplace=True)
        ret = pd.concat([ret, out_temp])

    return ret


def get_testing_flattened():
    df = get_testing()
    df = df.groupby(level = [0,2], axis = 1).mean()[['estimated']]
    df.columns = df.columns.droplevel().tolist()
    df.reset_index(inplace=True)
    df.rename(columns={'datetime': 'ds'}, inplace=True)
    df['weather_data_type'] = 'estimated'
    return df[['location', 'ds', 'weather_data_type'] + [i for i in df.columns.tolist() if i not in ['location', 'ds', 'y', 'weather_data_type']]]


#### Other helper functions

### 2. Avoiding Deductions

#### 2.1 Exploratory Data Analysis

##### 2.1.1 Search domain knowledge

##### 2.1.2 Check if the data is intuitive


##### 2.1.3 Understand how the data was generated 


##### 2.1.4 Explore individual features

##### 2.1.5 Explore pairs and groups of features 


##### 2.1.6 Clean up features 


#### 2.2 Predictors

Two examples of predictors used: 

1. Linear regression - see model 2

    Simple linear regression model using total radiation (direct + diffuse) as the independent variable and pv_measurement as the target variable.

2. XGBoost (eXtreme Gradient Boosting) - see model 4 and 5

     Uses gradient boosting to build an ensemble of decision trees, each trained on a subset of the data. The final prediction is a combination of the individual predictions from distinct trees.

#### 2.3 Feature Engineering

##### Feature Selection

Based on initial EDA, several of the original features were excluded:

- of the five fresh_snow columns, all except fresh_snow_3h:cm were discarded. This decision was based on the fact that 

In [ ]:

### Initial colums
"""
Index(['location', 'ds', 'y', 'weather_data_type', 'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
       'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
       'dew_point_2m:K', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W',
       'direct_rad_1h:J', 'effective_cloud_cover:p', 'elevation:m',
       'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
       'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'is_day:idx',
       'is_in_shadow:idx', 'msl_pressure:hPa', 'precip_5min:mm',
       'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
       'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
       'sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm',
       'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2',
       'sun_azimuth:d', 'sun_elevation:d', 'super_cooled_liquid_water:kgm2',
       't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
       'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
       'wind_speed_w_1000hPa:ms', 'total_rad_1h:J'],
      dtype='object')
"""
### Columns in cat_composite after feature selection
"""
self.common_features = ['sample_importance', 'is_estimated','dayofyear',
                                'is_day:idx',
                             'hour', 'month',
                            'total_rad_1h:J',
                            'sun_elevation:d',
                            'sun_azimuth:d',
                            'is_in_shadow:idx',
                            'effective_cloud_cover:p']
        
self.random_features = ['absolute_humidity_2m:gm3',
                                'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
                                'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
                                'dew_point_2m:K', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W',
                                'direct_rad_1h:J', 'fresh_snow_3h:cm',
                                'precip_5min:mm','precip_type_5min:idx', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
                                'sfc_pressure:hPa','snow_water:kgm2',
                                'super_cooled_liquid_water:kgm2',
                                't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
                                'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms']
"""                               


##### Feature Extraction

#### 2.4 Model Interpretation

### 3. Models
This section contains all model classes used to generate submitted Kaggle predictions. Models are ordered chronologically by the first date they were used to generate a submission.

#### MetaModel

Description: 
Abstract class which provides a framework for implementing subsequent models. MetaModel contains a test function common to all classes, which standardizes how models are evaluated (MAE-based). It also contains three abstract functions which are implemented in the actual models, namely preprocess, train, and predict.

In [ ]:
class MetaModel(ABC):     
    def __init__(self, model_name):
        self.model_name = model_name
        self.model = None

        return

    # Denne df-en må ha y som kolonne
    def test(self, df: pd.DataFrame, n_splits=5):
        """
            Expanding window cross-validation, df must have y in it for testing against predictions
        """
        print(f"Testing {self.model_name}")
        column_names = df.columns.tolist()
        if 'y' not in column_names:
            raise Exception(f"Missing observed y in columns. Available are {column_names}")

        # This is unecessary because we already clean it when calling train
        # drop_y_with_na
        df = df.dropna(subset=['y'], inplace=False)

        MAE_values = []
        MSE_values = []

        # tscv = TimeSeriesSplit(n_splits=n_splits)
        kf =KFold(n_splits=n_splits, shuffle=True, random_state=42)

        for train_index, test_index in kf.split(df):
            train_partition = df.iloc[train_index]
            valid_partition = df.iloc[test_index]

            self.train(train_partition)
            predictions = self.predict(valid_partition)
            
            y_true = valid_partition['y']
            y_pred = predictions['y_pred']

            MAE = mean_absolute_error(y_true, y_pred)
            MAE_values.append(MAE)

            MSE_values.append((y_pred - y_true).mean())

            print(f"Run {len(MAE_values)} MAE =", MAE)

        print("Mean Signed Error vals", MSE_values)
        average_mae = statistics.mean(MAE_values)
        print("MAE Vals: MEAN:", average_mae, 'ALL:' , MAE_values)
        
        return MAE_values
    
    @abstractmethod
    def preprocess(df: pd.DataFrame) -> pd.DataFrame:
        """
            Takes in single-index (datetime as index) df, and returns df with only desired features
        """    
        pass
    
    @abstractmethod
    def train(df: pd.DataFrame):
        """

        """
        pass

    @abstractmethod
    def predict(df: pd.DataFrame):
        """
            Runs trained model on on input df, preprocessing the df first and then returns datetime and y_pred
        """
        pass

#### 1. Naive estimator

Description: 
For each location, the model predicts pv_measurement to be the average pv_measurement for that location. 

In [ ]:
class NaiveModel(MetaModel):
    
    def __init__(self):
        super().__init__("Naive Model")
        
    def preprocess(self, df):
        """
        """
        temp_df = df.copy()

        return temp_df

    def train(self, df):
        """
        """
        df = self.preprocess(df)

        self.model = df.y.mean()


    def predict(self, df):
        df = self.preprocess(df)

        df['y_pred'] = self.model

        return df[['ds', 'y_pred']].copy()

#### 2. Linear Regression

In [ ]:
class LinearRegressionModel(MetaModel):
    
    def __init__(self):
        super().__init__("Linear Regression")
        
    def preprocess(self, df):
        """
        """
        temp_df = df.copy()

        temp_df['total_rad_1h:J'] = df['diffuse_rad_1h:J'] + df['direct_rad_1h:J']

        temp_df['total_rad_1h:J'].ffill(inplace=True)
        temp_df['total_rad_1h:J'].bfill(inplace=True)

        temp_df['total_rad_1h:J'].fillna(temp_df['total_rad_1h:J'].interpolate().cummax(), inplace=True)

        if('y' in temp_df.columns.tolist()):
            temp_df['y'].fillna(df['y'].interpolate().cummax(), inplace=True)

        return temp_df

    def train(self, df):
        """
        """
        df = self.preprocess(df)

        self.model = LinearRegression()
        self.model.fit(df['total_rad_1h:J'].values.reshape(-1, 1), df['y'].values.reshape(-1, 1))


    def predict(self, df):
        df = self.preprocess(df)

        y_preds = self.model.predict(df['total_rad_1h:J'].values.reshape(-1, 1))
        y_preds_arr = np.array(y_preds)

        out_df = pd.DataFrame(y_preds_arr, columns=["y_pred"])        

        return out_df

#### 3. Prophet

In [1]:
class ProphetModel(MetaModel):
    """
    Hyperparameters, default in parentheses:

        'C_observed_only' (False) - whether location C should only use observed data
        'features_by_corr0.6' (True) - use features with corr > 0.6 with y, if false uses 'total_rad_1h:J'
    """
    
    def __init__(self, hyperparameters = None):
        super().__init__("Prophet Model")
        self.hyperparameters = hyperparameters
        
    def preprocess(self, df):
        """
        """
        df = df.copy()

        if self.hyperparameters is not None and self.hyperparameters["C_observed_only"]:
            if location == "C":
                df = df[df.weather_data_type == 'observed'].copy()


        df['total_rad_1h:J'] = df['diffuse_rad_1h:J'] + df['direct_rad_1h:J']

        df = df.dropna(axis=0, how="all", subset="total_rad_1h:J")

        if('y' in df.columns.tolist()):
            df = df.dropna(axis=0, how="all", subset="y")

        df.fillna(0, inplace=True)
        
        return df

    def train(self, df):
        """
        """
        df = self.preprocess(df)

        self.prophet_model = Prophet()
        
        feat_find = df.corr().y.apply(abs).sort_values(ascending=False)
        features = feat_find[feat_find > 0.6].index.tolist()
        features.remove("y")
        
        if self.hyperparameters is not None and not self.hyperparameters["features_by_corr0.6"]:
            features = ['total_rad_1h:J']
        
        for feat in features:
            self.prophet_model.add_regressor(feat)

        self.prophet_model.fit(df)


    def predict(self, df):
        """
        """
        df = self.preprocess(df)

        forecast = self.prophet_model.predict(df)
        # self.prophet_model.plot_components(forecast)
        # fig = prophet_model.plot_components(forecast)
        temp_ret = forecast[['ds', 'yhat']].rename(columns={'yhat':'y_pred'})

        # force negative values to zero
        temp_ret.y_pred = temp_ret.y_pred.apply(lambda a : max(a, 0))

        return temp_ret

NameError: name '__file__' is not defined

#### 4 XG Boost Rev1

In [2]:
class XGBoostModel(MetaModel):

    def __init__(self):
        super().__init__("XGBoost")

    def preprocess(self, df):
        temp_df = df.copy()

        # SETTING ALL NAN TO 0 CONFORMING TO XGBOOST
        temp_df.fillna(0, inplace=True)

        print("NUM NANS AFTER FILLNA:", temp_df.isna().sum())


        temp_df['location'] = pd.factorize(temp_df['location'])[0]
        
        if 'diffuse_rad_1h:J' in df.columns and 'direct_rad_1h:J' in df.columns:
            temp_df['total_rad_1h:J'] = df['diffuse_rad_1h:J'] + df['direct_rad_1h:J']

        temp_df.dropna(axis=0, how="all", subset=["total_rad_1h:J"], inplace=True)

        if 'ds' in temp_df.columns:
            try:
                temp_df['ds'] = pd.to_datetime(temp_df['ds'])
                temp_df['hour_of_day'] = temp_df['ds'].dt.hour
                temp_df['month'] = temp_df['ds'].dt.month
            except:
                print("Error converting ds to datetime")

        if 'y' in temp_df.columns:
            temp_df.dropna(axis=0, how="all", subset=["y"], inplace=True)

        selected_cols = [
            'location', 
            'total_rad_1h:J',
            'hour_of_day', 
            'month', 
            'clear_sky_energy_1h:J', 
            'sun_elevation:d'
        ]

        if 'y' in df.columns:
            selected_cols.append('y')


        return temp_df[selected_cols]

    def train(self, df):
        df = self.preprocess(df)
        features = [col for col in df.columns if col != 'y']
        X = df[features]
        y = df['y']
        self.model = xgb.XGBRegressor()
        self.model.fit(X, y)

    def predict(self, df):
        df = self.preprocess(df)
        features = [col for col in df.columns if col != 'y']
        X = df[features]
        y_preds = self.model.predict(X)
        return pd.DataFrame({'y_pred': y_preds})

KeyboardInterrupt: 

#### 5. XG Boost Henrik

In [ ]:
class XGBoostHenrik(MetaModel):
    
    def __init__(self, features = None):
        super().__init__("XGBoost Henrik")
        self.features = []
        if features is not None:
            self.features = features  # Use the provided features if not None
        else:
            self.features.extend(['is_estimated', 'sample_importance',
                              'dayofyear',
                             'hour',
                            'total_rad_1h:J',
        'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
       'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
       'dew_point_2m:K', 'effective_cloud_cover:p', 'elevation:m',
       'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
       'fresh_snow_3h:cm', 'fresh_snow_6h:cm',
       'is_in_shadow:idx', 'msl_pressure:hPa', 'precip_5min:mm',
       'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
       'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
       'sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm',
       'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2',
       'sun_azimuth:d', 'sun_elevation:d', 'super_cooled_liquid_water:kgm2',
       't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
       'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
       'wind_speed_w_1000hPa:ms'])
            
        
    def preprocess(self, df: pd.DataFrame):
        """
        """
        temp_df = df.copy()

        has_target = 'y' in temp_df.columns        
        
        ##################################################################################### 
        # FEATURE ENGINEERING
        #####################################################################################

         # Emphasize test start-end: Starting date: 2023-05-01 00:00:00 Ending data 2023-07-03 23:00:00
        temp_df['sample_importance'] = 1
        temp_df.loc[(temp_df['ds'].dt.month >= 5) & 
                    (temp_df['ds'].dt.month < 7), 'sample_importance'] = 2
        
        temp_df.loc[(temp_df['ds'].dt.month == 7) &
                    (temp_df['ds'].dt.day <= 4), 'sample_importance'] = 2
        
        # Add is_estimated parameter
        temp_df['is_estimated'] = (temp_df['weather_data_type'] == 'estimated')
        temp_df['is_estimated'] = temp_df['is_estimated'].astype(int)

        temp_df['total_rad_1h:J'] = temp_df['diffuse_rad_1h:J'] + temp_df['direct_rad_1h:J']    
        
        # Extracting hour-of-day and month, and making them cyclical
        temp_df['hour'] = temp_df['ds'].dt.hour
        temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2

        temp_df['dayofyear'] = temp_df['ds'].dt.day_of_year
        temp_df['dayofyear'] = np.sin(2 * np.pi * (temp_df['dayofyear'] - 80)/ 365)
   
        # SETTING NAN TO 0 CONFORMING TO XGBOOST
        temp_df.fillna(0, inplace=True)

        #####################################################################################

        # DROPPING UNEEEDED FEATURES
        if(has_target):
            features_w_y = self.features + ['y']
            temp_df = temp_df[features_w_y]

        else:
            temp_df = temp_df[self.features]

        return temp_df

    def train(self, df):
        """
        """

        temp_df = self.preprocess(df)

        # Separate features and target
        X = temp_df.drop('y', axis=1, inplace=False).copy().values
        y = temp_df['y'].copy().values

        # Train test split
        #X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

        params = {
            'objective': "reg:absoluteerror",
            'eta': 0.25,
            'max_depth': 8,
            'lambda': 1
        }

        # Setup XGB
        self.model = xgb.XGBRegressor(**params)

        self.model.fit(
            X,
            y,
            verbose=True,
        )



    def predict(self, df):
        """
        """
        df = self.preprocess(df)

        features = [col for col in df.columns if col != 'y']
        X = df[features].values
        y_preds = self.model.predict(X)

        # Set all negative predictions to 0
        y_preds = np.maximum(y_preds, 0)

        out_df = pd.DataFrame(data={'y_pred': y_preds})

        return out_df

#### 6. Tree Composite 

In [ ]:
from xgboost_henrik import XGBoostHenrik
from catboost_henrik import CatBoostHenrik
from lightgbm_henrik import LightBGMHenrik

class TreeCompositeHenrik(MetaModel):
    
    def __init__(self):
        super().__init__("TreeComposite Henrik")
        self.features = []
        
        self.features.extend(['month',
                             'hour',
                            'total_rad_1h:J',
        'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
       'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
       'dew_point_2m:K', 'effective_cloud_cover:p', 'elevation:m',
       'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
       'fresh_snow_3h:cm', 'fresh_snow_6h:cm',
       'is_in_shadow:idx', 'msl_pressure:hPa', 'precip_5min:mm',
       'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
       'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
       'sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm',
       'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2',
       'sun_azimuth:d', 'sun_elevation:d', 'super_cooled_liquid_water:kgm2',
       't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
       'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
       'wind_speed_w_1000hPa:ms'])
                                    
        
    def preprocess(self, df: pd.DataFrame):
        """
        """
        temp_df = df.copy()

        has_target = 'y' in temp_df.columns        
        
        ##################################################################################### 
        # FEATURE ENGINEERING
        #####################################################################################

        temp_df['total_rad_1h:J'] = temp_df['diffuse_rad_1h:J'] + temp_df['direct_rad_1h:J']    
        
        # Extracting hour-of-day and month, and making them cyclical
        temp_df['hour'] = temp_df['ds'].dt.hour
        ml.utils.map_hour_to_seasonal(temp_df, 'hour')

        temp_df['month'] = temp_df['ds'].dt.month
        ml.utils.map_month_to_seasonal(temp_df, 'month')
   
        # SETTING NAN TO 0 CONFORMING TO XGBOOST
        temp_df.fillna(0, inplace=True)

        #####################################################################################

        # DROPPING UNEEEDED FEATURES
        if(has_target):
            features_w_y = self.features + ['y']
            temp_df = temp_df[features_w_y]

        else:
            temp_df = temp_df[self.features]

        return temp_df

    def train(self, df: MetaModel):
        """
        """

        self.models = {
            "XGBoost Henrik": XGBoostHenrik(),
            "CatBoost Henrik": CatBoostHenrik(),
            "LightGBM Henrik": LightBGMHenrik()
        }

        for key in self.models:
            self.models[key].train(df)



    def predict(self, df):
        """
        """
        
        all_preds = None

        out_df = None

        for key in self.models:
            y_pred = self.models[key].predict(df)['y_pred']
            if(all_preds is None):
                all_preds = pd.DataFrame(y_pred)
            else:
                all_preds[key] = y_pred.values

            
        avg_series = all_preds.mean(axis=1)

        print("The different models produced the following predictions:")
        print(all_preds)
        print("Averages")
        print(avg_series)

        return pd.DataFrame(avg_series, columns=['y_pred'])

    

#### 7. CatBoost Henrik

In [ ]:
class CatBoostHenrik(MetaModel):
    
    def __init__(self, features=None):
        super().__init__("CatBoost Henrik")
        if(features):
            self.features = features
        else:
            self.features = []
            self.features.extend(['sample_importance',
                                'dayofyear',
                                'hour',
                                'total_rad_1h:J',
            'absolute_humidity_2m:gm3',
            'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
            'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
            'dew_point_2m:K', 'effective_cloud_cover:p', 'elevation:m',
            'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
            'fresh_snow_3h:cm', 'fresh_snow_6h:cm',
            'is_in_shadow:idx', 'msl_pressure:hPa', 'precip_5min:mm',
            'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
            'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
            'sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm',
            'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2',
            'sun_azimuth:d', 'sun_elevation:d', 'super_cooled_liquid_water:kgm2',
            't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
            'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
            'wind_speed_w_1000hPa:ms'])
                           
    def preprocess(self, df: pd.DataFrame):
        """
        """
        temp_df = df.copy()

        has_target = 'y' in temp_df.columns        
        
        ##################################################################################### 
        # FEATURE ENGINEERING
        #####################################################################################

         # Emphasize test start-end: Starting date: 2023-05-01 00:00:00 Ending data 2023-07-03 23:00:00
        temp_df['sample_importance'] = 1
        temp_df.loc[(temp_df['ds'].dt.month >= 5) & 
                    (temp_df['ds'].dt.month < 7), 'sample_importance'] = 2
        
        temp_df.loc[(temp_df['ds'].dt.month == 7) &
                    (temp_df['ds'].dt.day <= 4), 'sample_importance'] = 2
        
        # Add is_estimated parameter
        temp_df['is_estimated'] = (temp_df['weather_data_type'] == 'estimated')
        temp_df['is_estimated'] = temp_df['is_estimated'].astype(int)

        temp_df['total_rad_1h:J'] = temp_df['diffuse_rad_1h:J'] + temp_df['direct_rad_1h:J']    
        
        # Extracting hour-of-day and month, and making them cyclical
        temp_df['hour'] = temp_df['ds'].dt.hour
        temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2

        temp_df['month'] = temp_df['ds'].dt.month
        temp_df['month'] = (np.sin(2 * np.pi * (temp_df['month'])/ 12) + 1) / 2


        temp_df['dayofyear'] = temp_df['ds'].dt.day_of_year
        temp_df['dayofyear'] = np.sin(2 * np.pi * (temp_df['dayofyear'] - 80)/ 365)
   
        # SETTING NAN TO 0 CONFORMING TO XGBOOST
        temp_df.fillna(0, inplace=True)

        #####################################################################################

        # DROPPING UNEEEDED FEATURES
        if(has_target):
            features_w_y = self.features + ['y']
            temp_df = temp_df[features_w_y]

        else:
            temp_df = temp_df[self.features]

        return temp_df

    def train(self, df):
        """
        """

        temp_df = self.preprocess(df)

        # Separate features and target
        X = temp_df.drop('y', axis=1, inplace=False).copy().values
        y = temp_df['y'].copy().values

        # Train test split
        #X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

        params = {
            'objective': "MAE",
            'learning_rate': 0.04,
            'depth': 9,
            'iterations': 7000,
            'logging_level': 'Silent'
        }

        
        # Setup XGB
        self.model = cb.CatBoostRegressor(**params)

              
        self.model.fit(
             X,
             y,
             verbose=True,
             sample_weight=temp_df['sample_importance']
        )
        
    def predict(self, df):
        """
        """
        df = self.preprocess(df)

        features = [col for col in df.columns if col != 'y']
        X = df[features].values
        y_preds = self.model.predict(X)

        # Set all negative predictions to 0
        y_preds = np.maximum(y_preds, 0)

        out_df = pd.DataFrame(data={'y_pred': y_preds})

        return out_df
    

#### 8. LightGBM Henrik

In [ ]:
class LightBGMHenrik(MetaModel):
    
    def __init__(self, features=None):
        super().__init__("LightBGM Henrik")

        self.features = []

        if features:
            self.features = features
        
        else:
            self.features.extend(['sample_importance',
                                'dayofyear',
                                'hour',
                                'total_rad_1h:J',
                                'is_day:idx',
            'absolute_humidity_2m:gm3',
            'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
            'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
            'dew_point_2m:K', 'effective_cloud_cover:p', 'elevation:m',
            'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
            'fresh_snow_3h:cm', 'fresh_snow_6h:cm',
            'is_in_shadow:idx', 'msl_pressure:hPa', 'precip_5min:mm',
            'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
            'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
            'sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm',
            'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2',
            'sun_azimuth:d', 'sun_elevation:d', 'super_cooled_liquid_water:kgm2',
            't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
            'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
            'wind_speed_w_1000hPa:ms'])
                                      
        
    def preprocess(self, df: pd.DataFrame):
        """
        """
        temp_df = df.copy()

        has_target = 'y' in temp_df.columns
        
        ##################################################################################### 
        # FEATURE ENGINEERING
        #####################################################################################

         # Emphasize test start-end: Starting date: 2023-05-01 00:00:00 Ending data 2023-07-03 23:00:00
        temp_df['sample_importance'] = 1
        temp_df.loc[(temp_df['ds'].dt.month >= 5) & 
                    (temp_df['ds'].dt.month < 7), 'sample_importance'] = 2
        
        temp_df.loc[(temp_df['ds'].dt.month == 7) &
                    (temp_df['ds'].dt.day <= 4), 'sample_importance'] = 2
        
        # Add is_estimated parameter
        temp_df['is_estimated'] = (temp_df['weather_data_type'] == 'estimated')
        temp_df['is_estimated'] = temp_df['is_estimated'].astype(int)

        temp_df['total_rad_1h:J'] = temp_df['diffuse_rad_1h:J'] + temp_df['direct_rad_1h:J']    
        
        # Extracting hour-of-day and month, and making them cyclical
        temp_df['hour'] = temp_df['ds'].dt.hour
        temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2

        temp_df['month'] = temp_df['ds'].dt.month
        temp_df['month'] = (np.sin(2 * np.pi * (temp_df['month'])/ 12) + 1) / 2

        temp_df['dayofyear'] = temp_df['ds'].dt.day_of_year
        temp_df['dayofyear'] = np.sin(2 * np.pi * (temp_df['dayofyear'] - 80)/ 365)
   
        # SETTING NAN TO 0 CONFORMING TO LGBM
        temp_df.fillna(0, inplace=True)
    
        #####################################################################################

        # DROPPING UNEEEDED FEATURES
        if(has_target):
            features_w_y = self.features + ['y']
            temp_df = temp_df[features_w_y]

        else:
            temp_df = temp_df[self.features]

        return temp_df

    def train(self, df):
        """
        """

        temp_df = self.preprocess(df)

        # Separate features and target
        sample_importance = temp_df['sample_importance']
        X = temp_df.drop(['y','sample_importance'], axis=1, inplace=False).copy().values
        y = temp_df['y'].copy().values

        # Train test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=42)

        num_iterations = 2000
        early_stopping_percentage = 0.1

        params = {
            'objective': "mae",
            'learning_rate': 0.1,
            'num_leaves': 128,
            'zero_as_missing': True,
            'num_iterations': num_iterations
        }


        # Setup XGB
        self.model = lgbm.LGBMRegressor(**params)

        self.model.fit(
            X_train,
            y_train,
            eval_set = (X_test, y_test),
            #sample_weight = sample_importance,
            eval_metric = 'mae',
            early_stopping_rounds = round(num_iterations * early_stopping_percentage)
        )



    def predict(self, df):
        """
        """
        df = self.preprocess(df)

        features = [col for col in df.columns if col not in ['y', 'sample_importance']]
        X = df[features].values
        y_preds = self.model.predict(X)

        # Set all negative predictions to 0
        y_preds = np.maximum(y_preds, 0)

        out_df = pd.DataFrame(data={'y_pred': y_preds})

        return out_df

#### 9. AutoML H20

In [ ]:
### ADD MODEL CODE HERE

#### 10. Autogluon Henrik

In [ ]:
class AutoGluonHenrik(MetaModel):
    
    def __init__(self, time_limit=None, excluded_models: list = []):
        super().__init__("AutoGluon Henrik")

        self.time_limit = time_limit
        self.excluded_models = excluded_models

        self.features = []
        
        self.features.extend(['sample_importance',
                            'total_rad_1h:J',
        'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
       'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
       'dew_point_2m:K', 'effective_cloud_cover:p', 'elevation:m',
       'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
       'fresh_snow_3h:cm', 'fresh_snow_6h:cm',
       'is_in_shadow:idx', 'msl_pressure:hPa', 'precip_5min:mm',
       'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
       'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
       'sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm',
       'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2',
       'sun_azimuth:d', 'sun_elevation:d', 'super_cooled_liquid_water:kgm2',
       't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
       'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
       'wind_speed_w_1000hPa:ms'])
        
        """

        self.features.extend(['total_rad_1h:J', 'month', 'hour', 'sun_elevation:d', 'effective_cloud_cover:p'])
        """

                                  
        

    def preprocess(self, df: pd.DataFrame):
        """
        """
        temp_df = df.copy()

        has_target = 'y' in df.columns

        if has_target:
            temp_df = temp_df[temp_df['y'].notna()]

        ##################################################################################### 
        # FEATURE ENGINEERING
        #####################################################################################

        # Emphasize test start-end: Starting date: 2023-05-01 00:00:00 Ending data 2023-07-03 23:00:00
        temp_df['sample_importance'] = 1
        temp_df.loc[(temp_df['ds'].dt.month >= 5) & 
                    (temp_df['ds'].dt.month < 7), 'sample_importance'] = 2
        
        temp_df.loc[(temp_df['ds'].dt.month == 7) &
                    (temp_df['ds'].dt.day <= 4), 'sample_importance'] = 2
        
        # Add is_estimated parameter
        temp_df['is_estimated'] = (temp_df['weather_data_type'] == 'estimated')
        temp_df['is_estimated'] = temp_df['is_estimated'].astype(int)

        temp_df['total_rad_1h:J'] = temp_df['diffuse_rad_1h:J'] + temp_df['direct_rad_1h:J']    
        
        # Extracting hour-of-day and month, and making them cyclical
        temp_df['hour'] = temp_df['ds'].dt.hour
        temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2

        temp_df['dayofyear'] = temp_df['ds'].dt.day_of_year
        temp_df['dayofyear'] = np.sin(2 * np.pi * (temp_df['dayofyear'] - 80)/ 365)
   
        # SETTING NAN TO 0 CONFORMING TO XGBOOST
        temp_df.fillna(0, inplace=True)

        #####################################################################################
        if(has_target):
            
            return temp_df[self.features + ['y']]
        else:
            return temp_df[self.features]

    def train(self, df):
        """
        """
        temp_df = self.preprocess(df)


        train_data = TabularDataset(temp_df)

        self.model = TabularPredictor(
            label='y',
            eval_metric='mean_absolute_error',
            problem_type='regression',
            sample_weight='sample_importance',
            weight_evaluation=True
        ).fit(train_data,
              time_limit = self.time_limit,
              excluded_model_types = self.excluded_models,
              presets=['good_quality']
              )


    def predict(self, df):
        """
        """
        df = self.preprocess(df)

        features = [col for col in df.columns if col != 'y']
        X = df[features]

        y_preds = self.model.predict(X)
        print("AUTOGLUON MODEL OVERVIEW:")
        print(self.model.get_model_names())
       

        out_df = pd.DataFrame(data={'y_pred': y_preds})

        return out_df

#### 11. LGBM Composite

In [ ]:
from lightgbm_henrik import LightBGMHenrik

class LGBMCompositeHenrik(MetaModel):
    
    def __init__(self, num_models=10):
        super().__init__("LGBMComposite Henrik")

        self.num_models = num_models
        self.common_features = ['sample_importance', 'is_estimated','dayofyear',
                                'is_day:idx',
                             'hour', 'month',
                            'total_rad_1h:J',
                            'sun_elevation:d',
                            'sun_azimuth:d',
                            'is_in_shadow:idx',
                            'effective_cloud_cover:p']
        
        self.random_features = ['absolute_humidity_2m:gm3',
                                'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
                                'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
                                'dew_point_2m:K', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W',
                                'direct_rad_1h:J', 'fresh_snow_3h:cm',
                                'precip_5min:mm','precip_type_5min:idx', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
                                'sfc_pressure:hPa','snow_water:kgm2',
                                'super_cooled_liquid_water:kgm2',
                                't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
                                'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms']
                           
        
    def preprocess(self, df: pd.DataFrame):

        return df

    def train(self, df: pd.DataFrame):
        df = self.preprocess(df)
        num_rand_features = round(len(self.random_features) * 0.85)

        features = dict()

        self.models = dict()

        for i in range(self.num_models):
            temp_rand_features = random.sample(self.random_features, num_rand_features)
            features[i] = self.common_features + temp_rand_features
            self.models[f'LGBM_{i} Henrik'] = LightBGMHenrik(features = features[i])

            print(f'LGBM_{i} Henrik')

        for key in self.models:
            self.models[key].train(df)



    def predict(self, df):
        """
        """
        
        all_preds = None

        for key in self.models:
            y_pred = self.models[key].predict(df)['y_pred']
            if(all_preds is None):
                all_preds = pd.DataFrame(y_pred)
            else:
                all_preds[key] = y_pred.values

            
        avg_series = all_preds.mean(axis=1)

        avg_series = np.maximum(avg_series, 0)

        return pd.DataFrame(avg_series, columns=['y_pred'])

#### 12. Composite Composite

In [ ]:
from models_here_hahah.cat_composite import CatCompositeHenrik
from xg_boost_composite import XGBoostComposite
from autogluon_henrik import AutoGluonHenrik, AutoGluonJacob
from catboost_henrik import CatBoostHenrik

class CompositeCompositeHenrik(MetaModel):
    
    def __init__(self):
        super().__init__("CompositeComposite Henrik")

        self.common_features = ['dayofyear',
                             'hour',
                            'total_rad_1h:J',
                            'sun_elevation:d',
                            'sun_azimuth:d',
                            'is_in_shadow:idx',
                            'effective_cloud_cover:p']
        
        self.random_features = ['absolute_humidity_2m:gm3',
                            'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
                            'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
                            'dew_point_2m:K', 'elevation:m',
                            'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
                            'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'msl_pressure:hPa', 'precip_5min:mm',
                            'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
                            'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
                            'sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm',
                            'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2', 'super_cooled_liquid_water:kgm2',
                            't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
                            'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
                            'wind_speed_w_1000hPa:ms']
                                    
        
    def preprocess(self, df: pd.DataFrame):

        return df.copy()

    def train(self, df: MetaModel):
        
        # Excluded AutoGluon models (https://auto.gluon.ai/0.4.0/api/autogluon.predictor.html)
        excluded_ag_models = None ### NOT GOOD!!!!!!! DONT EXCLUDEEEE

        df = df.copy()

        df.drop(df[(df['ds'].dt.month > 9) & (df['ds'].dt.month < 4)].index, inplace = True)

        self.models = {
            #"XGBoost Composite": XGBoostComposite(),
            #"AutoGluon 5min": AutoGluonHenrik(time_limit=60*30, excluded_models=excluded_ag_models),
            "AutoGluonJacob 10min": AutoGluonJacob(time_limit=60*10),
            "CatComposite x20": CatCompositeHenrik(num_models=20)
        }

        for key in self.models:
            self.models[key].train(df)



    def predict(self, df):
        """
        """
        
        all_preds = None

        out_df = None

        for key in self.models:
            y_pred = self.models[key].predict(df)['y_pred']
            if(all_preds is None):
                all_preds = pd.DataFrame(y_pred)
            else:
                all_preds[key] = y_pred.values

            
        avg_series = all_preds.mean(axis=1)

        print("The different models produced the following predictions:")
        print(all_preds)

        avg_series = np.maximum(avg_series, 0)

        return pd.DataFrame(avg_series, columns=['y_pred'])

#### 13. XG Boost Composite

In [ ]:
class XGBoostComposite(MetaModel):
    
    def __init__(self):
        super().__init__("XGBoost Composite")

        self.common_features = ['is_estimated','dayofyear',
                             'hour',
                            'total_rad_1h:J',
                            'sun_elevation:d',
                            'sun_azimuth:d',
                            'is_in_shadow:idx',
                            'effective_cloud_cover:p']
        
        self.random_features = ['absolute_humidity_2m:gm3',
                            'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
                            'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
                            'dew_point_2m:K', 'elevation:m',
                            'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
                            'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'msl_pressure:hPa', 'precip_5min:mm',
                            'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
                            'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
                            'sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm',
                            'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2', 'super_cooled_liquid_water:kgm2',
                            't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
                            'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
                            'wind_speed_w_1000hPa:ms']
                    
        
    def preprocess(self, df: pd.DataFrame):
        return df.copy()

    
    def train(self, df: pd.DataFrame, use_meta_learner=True):
        num_models = 30
        num_rand_features = round(len(self.random_features) * 0.8)  
        df = df.copy()
        df['month'] = df['ds'].dt.month

        meta_train_df = df[(df['month'] == 5) | (df['month'] == 6) | (df['month'] == 7)].sample(frac=0.5)
        print("Meta-train % of full DF", len(meta_train_df)/len(df))
        train_df = df.loc[~df.index.isin(meta_train_df)]

        features = dict()
        self.models = dict()

        for i in range(num_models):
            temp_rand_features = random.sample(self.random_features, num_rand_features)
            features[i] = self.common_features + temp_rand_features
            self.models[f'XGBOOST_{i}'] = XGBoostHenrik(features = features[i])

        for key in self.models:
            print("Training model", key)
            self.models[key].train(train_df)
        
        if (use_meta_learner):                        
            y_preds = self.predict(meta_train_df, meta_training=True)

            self.meta_learner = LinearRegression(fit_intercept=False, positive=True)
            self.meta_learner.fit(y_preds, meta_train_df['y'])

            # Adjust weights so all are non-zero and positive
            """
            new_coefficients = np.copy(self.meta_learner.coef_) + 1/num_models
            new_coefficients = new_coefficients / new_coefficients.min()
            new_coefficients = new_coefficients / new_coefficients.sum()
            self.meta_learner.coef_ = new_coefficients
            """

            self.meta_learner.coef_ = np.full(shape = self.meta_learner.coef_.shape, fill_value = 1/num_models)

            print(self.meta_learner.coef_)
    
    def predict(self, df, meta_training = False):
        """
        """
        all_preds = None
        out_df = None
        for key in self.models:
            y_pred = self.models[key].predict(df)['y_pred']
            if(all_preds is None):
                all_preds = pd.DataFrame(y_pred)
            else:
                all_preds[key] = y_pred.values

        if (meta_training):
            print("RETURNING ALL_PREDS")
            return pd.DataFrame(all_preds)

        #print("THIS HAS GONE TOO FAR!")

        # Use meta-learner to calculate final output

        out_np = self.meta_learner.predict(all_preds)
        #print(out_np)

        """
        out_np = all_preds.mean(axis=1)

        print("The different models produced the following predictions:")
        print(out_np)

        out_np = np.maximum(out_np, 0)
        """
        return pd.DataFrame(out_np, columns=['y_pred'])

#### 14. Autogluon Jacob

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor

class AutoGluonJacob(MetaModel):
    
    def __init__(self, time_limit=60*3):
        super().__init__("AutoGluon Jacob")

        self.time_limit = time_limit

        # autogluon features
        # TabularPredictor (usage : **params_TabularPredictor)
        self.params_TabularPredictor = \
            {
                'label': 'y',
                'problem_type': 'regression', 
                'eval_metric': 'mean_absolute_error',
                'verbosity': 2,
            } 
        # TabularPredictor.fit
        self.params_TabularPredictor_fit = \
            {
                'time_limit': self.time_limit,
                'presets': 'high_quality', # [‘best_quality’, ‘high_quality’, ‘good_quality’, ‘medium_quality’, ‘optimize_for_deployment’, ‘interpretable’, ‘ignore_text’]
                #'hyperparameters': 'default',
                # 'auto_stack': False,
                # 'num_bag_folds': None, # set automatically by auto_stack True
                # 'num_bag_sets': None, # set to 20 because of auto_stack
                # 'num_stack_levels': None, # set automatically by auto_stack True
                'hyperparameter_tune_kwargs': 'auto', # None to disable
                # 'refit_full': True,
                # 'feature_prune_kwargs': {}, # If None, do not perform feature pruning. If empty dictionary, perform feature pruning with default configurations.
            }

        self.use_tuning_data = True # 'sample_weight', 'random'
        self.use_sample_weight = True

        if self.use_sample_weight: # auto_weight a feature that exists
            self.params_TabularPredictor['sample_weight'] = 'sample_importance'


        self.common_features = ['sample_importance', 'weather_data_type','dayofyear',
                                'is_day:idx',
                             'hour', 'month',
                            'total_rad_1h:J',
                            'sun_elevation:d',
                            'sun_azimuth:d',
                            'is_in_shadow:idx',
                            'effective_cloud_cover:p']
        
        self.random_features = ['absolute_humidity_2m:gm3',
                                'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
                                'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
                                'dew_point_2m:K', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W',
                                'direct_rad_1h:J', 'fresh_snow_3h:cm',
                                'precip_5min:mm','precip_type_5min:idx', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
                                'sfc_pressure:hPa','snow_water:kgm2',
                                'super_cooled_liquid_water:kgm2',
                                't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
                                'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms']
        

    def preprocess(self, df: pd.DataFrame):
        """
        """
        temp_df = df.copy()

        temp_df['total_rad_1h:J'] = temp_df['diffuse_rad_1h:J'] + temp_df['direct_rad_1h:J']    
        
        # Extracting hour-of-day and month, and making them cyclical
        temp_df['hour'] = temp_df['ds'].dt.hour
        temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2

        temp_df['dayofyear'] = temp_df['ds'].dt.day_of_year
        temp_df['dayofyear'] = np.sin(2 * np.pi * (temp_df['dayofyear'] - 80)/ 365)

        # temp_df['year'] = temp_df['ds'].dt.hour
        temp_df['month'] = temp_df['ds'].dt.month
        # temp_df['day'] = temp_df['ds'].dt.day
        # temp_df['dayofweek'] = temp_df['ds'].dt.dayofweek

        if self.use_sample_weight:
            # Emphasize test start-end: Starting date: 2023-05-01 00:00:00 Ending data 2023-07-03 23:00:00
            temp_df['sample_importance'] = 1
            temp_df.loc[(temp_df['ds'].dt.month >= 5) & 
                        (temp_df['ds'].dt.month < 7), 'sample_importance'] = 2
            
            temp_df.loc[(temp_df['ds'].dt.month == 7) &
                         (temp_df['ds'].dt.day <= 4), 'sample_importance'] = 2



        return temp_df.drop(columns=['ds'])[self.common_features + self.random_features + (['y'] if 'y' in temp_df else [])].copy()

    def train(self, df):
        """
        """
        print("Training JacobGluon")
        temp_df = self.preprocess(df)

        # temp_df.drop(df[(df['ds'].dt.month > 9) & (df['ds'].dt.month < 4)].index, inplace = True)


        if self.use_tuning_data:

            tuning_data = temp_df[(temp_df['month'] == 5) | (temp_df['month'] == 6)].sample(frac=0.5, random_state=42)
            train_data = TabularDataset(temp_df[~temp_df.isin(tuning_data.to_dict(orient='list')).all(1)])

            self.model = TabularPredictor(**self.params_TabularPredictor).fit(train_data, tuning_data=tuning_data, use_bag_holdout=True, **self.params_TabularPredictor_fit)
        else:
            train_data = TabularDataset(temp_df)

            self.model = TabularPredictor(**self.params_TabularPredictor).fit(train_data, **self.params_TabularPredictor_fit)

    def predict(self, df):
        """
        """
        df = self.preprocess(df)

        features = [col for col in df.columns if col != 'y']
        X = df[features]

        y_preds = self.model.predict(X)
        print("AUTOGLUON MODEL OVERVIEW:")
        print(self.model.leaderboard())       

        out_df = pd.DataFrame(data={'y_pred': y_preds})

        return out_df

ModuleNotFoundError: No module named 'autogluon'

#### 15. CatBoost Composite

In [ ]:
from catboost_henrik import CatBoostHenrik

class CatCompositeHenrik(MetaModel):
    
    def __init__(self, num_models=10):
        super().__init__("CatComposite Henrik")

        self.num_models = num_models
        # self.common_features = ['sample_importance', 'is_estimated','dayofyear',
        #                         'is_day:idx',
        #                      'hour', 'month',
        #                     'total_rad_1h:J',
        #                     'sun_elevation:d',
        #                     'sun_azimuth:d',
        #                     'is_in_shadow:idx',
        #                     'effective_cloud_cover:p']
        
        # self.random_features = ['absolute_humidity_2m:gm3',
        #                     'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
        #                     'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
        #                     'dew_point_2m:K',
        #                     'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
        #                     'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'msl_pressure:hPa', 'precip_5min:mm',
        #                     'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
        #                     'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
        #                     'sfc_pressure:hPa', 'snow_depth:cm',
        #                     'snow_water:kgm2', 'super_cooled_liquid_water:kgm2',
        #                     't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
        #                     'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms']
        
        self.common_features = ['sample_importance', 'is_estimated','dayofyear',
                                'is_day:idx',
                             'hour', 'month',
                            'total_rad_1h:J',
                            'sun_elevation:d',
                            'sun_azimuth:d',
                            'is_in_shadow:idx',
                            'effective_cloud_cover:p']
        
        self.random_features = ['absolute_humidity_2m:gm3',
                                'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
                                'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
                                'dew_point_2m:K', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W',
                                'direct_rad_1h:J', 'fresh_snow_3h:cm',
                                'precip_5min:mm','precip_type_5min:idx', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
                                'sfc_pressure:hPa','snow_water:kgm2',
                                'super_cooled_liquid_water:kgm2',
                                't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
                                'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms']
        """
        # FEATURES WITHOUT SNOW
        self.random_features = ['absolute_humidity_2m:gm3',
                            'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
                            'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
                            'dew_point_2m:K', 'elevation:m',
                            'msl_pressure:hPa', 'precip_5min:mm',
                            'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
                            'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
                            'sfc_pressure:hPa',
                            'super_cooled_liquid_water:kgm2',
                            't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
                            'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
                            'wind_speed_w_1000hPa:ms']
        """         
        
    def preprocess(self, df: pd.DataFrame):
        return df.copy()

    
    def train(self, df: pd.DataFrame, use_meta_learner=True):
        num_models = self.num_models
        num_rand_features = round(len(self.random_features) * 1)
        df = df.copy()
        df['month'] = df['ds'].dt.month

        # random_states = [41, 61, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]#[i for i in range(num_models - 1)]
        random_states = [i for i in range(23, 43)]


        meta_train_df = df[(df['month'] == 5) | (df['month'] == 6) | (df['month'] == 7)].sample(frac=0.5)
        print("Meta-train % of full DF", len(meta_train_df)/len(df))

        # DUPLICATE MONTHS WE PREDICT
        selected_months = df[(df['month'] == 5) | (df['month'] == 6) | (df['month'] == 7)].copy()
        train_df = pd.concat([df, selected_months], ignore_index=True)

        features = dict()
        self.models = dict()

        for i in range(num_models):
            temp_rand_features = random.sample(self.random_features, num_rand_features)
            features[i] = self.common_features + temp_rand_features
            self.models[f'CATBOOST_{i}'] = CatBoostHenrik(features = features[i], random_state=random_states[i])

        for key in self.models:
            print("Training model", key)
            self.models[key].train(train_df)
        
        """
        if (use_meta_learner):                        
            y_preds = self.predict(meta_train_df, meta_training=True)

            self.meta_learner = LinearRegression(fit_intercept=False, positive=True)
            self.meta_learner.fit(y_preds, meta_train_df['y'])

            # Adjust weights so all are non-zero and positive
            new_coefficients = np.copy(self.meta_learner.coef_) + 1/num_models
            new_coefficients = new_coefficients / new_coefficients.min()
            new_coefficients = new_coefficients / new_coefficients.sum()
            self.meta_learner.coef_ = new_coefficients

            print(self.meta_learner.coef_)
        """
    
    def predict(self, df, meta_training = False):

        all_preds = None
        out_df = None
        for key in self.models:
            y_pred = self.models[key].predict(df)['y_pred']
            if(all_preds is None):
                all_preds = pd.DataFrame(y_pred)
            else:
                all_preds[key] = y_pred.values

        if (meta_training):
            print("RETURNING ALL_PREDS")
            return pd.DataFrame(all_preds)

        """
        #print("THIS HAS GONE TOO FAR!")

        # Use meta-learner to calculate final output (DISABLED)
        out_np = self.meta_learner.predict(all_preds)
        #print(out_np)
        """
        out_np = all_preds.mean(axis=1)

        print("The different models produced the following predictions:")
        print(out_np)

        out_np = np.maximum(out_np, 0)

        return pd.DataFrame(out_np, columns=['y_pred'])

#### 91. FeedForward Neural Network

In [ ]:
class FeedforwardNNHenrik(MetaModel):
    
    def __init__(self):
        super().__init__("Feedforward Neural Network Henrik")
        self.features = []

        """
        self.features.extend(['month',
                             'hour',
                            'total_rad_1h:J',
                            'sun_elevation:d'])
        """
        
        self.features.extend(['month',
                             'hour',
                            'total_rad_1h:J',
                            'fresh_snow_12h:cm',
                            'snow_water:kgm2',
                            'is_day:idx',
                            'is_in_shadow:idx',
                            'rain_water:kgm2',
                            'sun_azimuth:d',
                            'sun_elevation:d',
                            't_1000hPa:K',
                            'dew_or_rime:idx',
                            'air_density_2m:kgm3',
                            'absolute_humidity_2m:gm3'])
        
        """
        self.features.extend(['super_cooled_liquid_water:kgm2',
                              'effective_cloud_cover:p', 'elevation:m',
                              'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
                              'msl_pressure:hPa', 'precip_5min:mm', 'prob_rime:p', 
                              'relative_humidity_1000hPa:p', 'visibility:m'
                              ])
        """                              
        
    def preprocess(self, df: pd.DataFrame):
        """
        """
        temp_df = df.copy()

        has_target = 'y' in temp_df.columns        
        
        ##################################################################################### 
        # FEATURE ENGINEERING
        #####################################################################################

        temp_df['total_rad_1h:J'] = temp_df['diffuse_rad_1h:J'] + temp_df['direct_rad_1h:J']    
        
        # Extracting hour-of-day and month, and making them cyclical
        temp_df['hour'] = temp_df['ds'].dt.hour
        ml.utils.map_hour_to_seasonal(temp_df, 'hour')

        temp_df['month'] = temp_df['ds'].dt.month
        ml.utils.map_month_to_seasonal(temp_df, 'month')

        # Make NaN interpolated
        ml.utils.interpolate_na(temp_df, self.features)


        # Normalize the features
        scaler = preprocessing.MinMaxScaler()
        temp_df[self.features] = scaler.fit_transform(temp_df[self.features])

        #####################################################################################

        # DROPPING UNEEEDED FEATURES
        if(has_target):
            features_w_y = self.features + ['y']
            temp_df = temp_df[features_w_y]

        else:
            temp_df = temp_df[self.features]

        return temp_df

    def train(self, df):
        """
        """

        temp_df = self.preprocess(df)

        # Separate features and target
        X = temp_df.drop('y', axis=1, inplace=False).copy().values
        y = temp_df['y'].copy().values

        # Train test split
        #X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

        
        print('Num features:', len(self.features))

        # Setup XGB
        self.model = models.Sequential()
        """
        self.model.add(layers.Dense(input_dim=len(self.features),
                       units=len(self.features), 
                       activation="relu"))
        
        self.model.add(layers.Dense(input_dim=len(self.features),
                       units=32, 
                       activation="relu"))
        
        self.model.add(layers.Dense(input_dim=32,
                       units=32, 
                       activation="relu"))


        self.model.add(layers.Dense(input_dim=32,
                       units=32, 
                       activation="relu"))
        
        self.model.add(layers.Dense(input_dim=32,
                       units=32, 
                       activation="relu"))
        

        self.model.add(layers.Dense(input_dim=32,
                       units=16, 
                       activation="relu"))
        """

        # Params
        learning_rate = 0.35
        

        # add the output layer
        self.model.add(layers.Dense(input_dim=len(self.features),
                            units=1,
                            activation='relu'))

        # define our loss function and optimizer
        self.model.compile(loss='mean_absolute_error',
                    # Adam is a kind of gradient descent
                    optimizer=optimizers.SGD(learning_rate=learning_rate))


        self.model.fit(
            X,
            y,
            verbose=True,
            epochs=200,
            batch_size=32
        )



    def predict(self, df):
        """
        """
        df = self.preprocess(df)

        features = [col for col in df.columns if col != 'y']
        X = df[features].values
        y_preds = self.model.predict(X)

        # Set all negative predictions to 0
        y_preds = np.maximum(y_preds, 0)

        # Make y_preds 1D
        y_preds = np.ravel(y_preds)

        out_df = pd.DataFrame(data={'y_pred': y_preds})

        return out_df

#### 92. LSTM

In [ ]:
class LSTM(MetaModel):
    def __init__(self, window_len):
        super().__init__("LSTM")

        self.keep_columns = ['total_rad_1h:J',
                        'fresh_snow_12h:cm',
                        'snow_water:kgm2',
                        'is_day:idx',
                        'is_in_shadow:idx',
                        'rain_water:kgm2',
                        'sun_azimuth:d',
                        'sun_elevation:d',
                        't_1000hPa:K',
                        'dew_or_rime:idx',
                        'air_density_2m:kgm3',
                        'absolute_humidity_2m:gm3',
                        'y'
        ]

        self.window_len = window_len
        self.batch_size = 1
        self.num_features = None
        self.epochs = 1
        self.scaler = None
    
    def test(self, df, n_splits=5):
        print(f"Testing {self.model_name}")
        column_names = df.columns.tolist()
        if 'y' not in column_names:
            raise Exception(f"Missing observed y in columns. Available are {column_names}")

        # This is unecessary because we already clean it when calling train
        #df_cleaned = self.preprocess(df)

        tscv = TimeSeriesSplit(n_splits=n_splits)

        MAE_values = []

        for train_index, test_index in tscv.split(df):

            train_partition = df.iloc[train_index]
            valid_partition = df.iloc[test_index]

            # Needs to be adjusted due to window length
            adj_valid_partition = pd.concat([train_partition[-self.window_len:], valid_partition], ignore_index=True)

            self.train(train_partition)
            predictions = self.predict(adj_valid_partition)
            
            y_true = valid_partition['y']
            y_pred = predictions['y_pred']

            MAE = mean_absolute_error(y_true, y_pred)
            MAE_values.append(MAE)

            print(f'Train-Test ratio:{len(train_partition)/len(valid_partition)} achieved MAE {MAE}')

        return MAE_values
        
    def preprocess(self, df, has_target_col=False):
        temp_df = df.copy()

        temp_df['total_rad_1h:J'] = df['diffuse_rad_1h:J'] + df['direct_rad_1h:J']

        # Only keep the columns in keep_columns that df actually has
        keep_columns_exist = [col for col in self.keep_columns if col in temp_df.columns]
        temp_df = temp_df[keep_columns_exist]
        
        # Clean out NaNs/NONEs -- May need to remove for more than just total_rad_1h:J
        #temp_df['total_rad_1h:J'].fillna(df['total_rad_1h:J'].interpolate().cummax())
        #temp_df = temp_df.dropna(axis=0, how="all", subset="total_rad_1h:J")

        if(has_target_col):
            temp_df['y'].fillna(df['y'].interpolate().cummax())
            # temp_df = temp_df.dropna(axis=0, how="all", subset="y")

        # Min-max scale all columns so all values in [0, 1]
        self.scaler = StandardScaler()
        temp_np = self.scaler.fit_transform(temp_df)

        # Convert to time-series of given window-lengths 
        # (Sequence_length is how long each time-window is, sequence_stride is how long the window shifts forward in time for each sequence)
        if(has_target_col):
            self.num_features = temp_np.shape[1] - 1

            features = temp_np[:, : self.num_features]
            targets = temp_np[:, self.num_features]

            temp_df = TimeseriesGenerator(data=features, targets=targets, length=self.window_len, shuffle=False, batch_size=self.batch_size)
        
        else:
            self.num_features = temp_np.shape[1]

            features = temp_np[:, : self.num_features]
            dummy_targets = np.zeros(features.shape)

            temp_df = TimeseriesGenerator(data=features, targets=dummy_targets, length=self.window_len, shuffle=False, batch_size=self.batch_size)
        
        # Returns a generator (keras datatype)
        return temp_df

    def train(self, df):
        # Make train-test split
        train_end = round(len(df) * 0.7)
        # test_df = df.iloc[train_end:]

        train_generator = self.preprocess(df, has_target_col = True)
        # test_generator = self.preprocess(test_df, has_target_col=True)

        # Design the model
        self.model = tf.keras.Sequential()

        self.model.add(tf.keras.layers.LSTM(64, input_shape=(self.window_len, self.num_features), return_sequences=True))
        self.model.add(tf.keras.layers.LeakyReLU(alpha=0.5))

        self.model.add(tf.keras.layers.LSTM(64, return_sequences=True))
        self.model.add(tf.keras.layers.LeakyReLU(alpha=0.5))

        self.model.add(tf.keras.layers.Dropout(0.3))
        self.model.add(tf.keras.layers.LSTM(32, return_sequences=False))

        self.model.add(tf.keras.layers.Dropout(0.3))
        self.model.add(tf.keras.layers.Dense(1))

        # Add pruning measures
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
        self.model.compile(loss=tf.losses.MeanSquaredError(), optimizer=tf.optimizers.Adam(), metrics=tf.metrics.MeanAbsoluteError())

        # Train the model
        history = self.model.fit(train_generator, epochs=self.epochs, shuffle=False, callbacks=[early_stopping])


    def predict(self, df: pd.DataFrame):
        """
        """
        output_generator = self.preprocess(df, ('y' in df.columns))
        model_out = self.model.predict(output_generator)

        print(model_out)
        print(model_out.shape)

        ### RESCALE BACK TO ORIGINAL SIZE
        temp_scaler = StandardScaler()
        index_of_y = self.keep_columns.index("y")
        temp_scaler.mean_, temp_scaler.scale_ = self.scaler.mean_[index_of_y], self.scaler.scale_[index_of_y]

        scaled_model_out = temp_scaler.inverse_transform(model_out)
        print(scaled_model_out)

        data = {'y_pred': scaled_model_out.flatten()}
        out_df = pd.DataFrame(data)

        if(len(df) != len(out_df) + self.window_len):
            print("INPUT DF WAS CLEANED S.T. INDEXES WONT MATCH UP")
            print("Original df:", len(df), "Output pred_df", len(out_df))

        return out_df
    

#### 93. PCA Regression

In [ ]:
class PCARegression(MetaModel):
    
    def __init__(self, pca_dimensions: int = 6):
        super().__init__("PCA Regression")
        self.keep_columns = ['ds',
                            'total_rad_1h:J',
                            'fresh_snow_12h:cm',
                            'snow_water:kgm2',
                            'is_day:idx',
                            'is_in_shadow:idx',
                            'rain_water:kgm2',
                            'sun_azimuth:d',
                            'sun_elevation:d',
                            't_1000hPa:K',
                            'dew_or_rime:idx',
                            'air_density_2m:kgm3',
                            'absolute_humidity_2m:gm3',
                            'y'
        ]

        self.pca_dimensions = pca_dimensions
    
    def interpolate_na(self, df, cols):
        for col in cols:
            df[col].ffill(inplace=True)
            df[col].bfill(inplace=True)

            df[col].fillna(df[col].interpolate().cummax(), inplace=True)            
        
    def preprocess(self, df):
        """
        """
        temp_df = df.copy()

        has_target_col = 'y' in temp_df.columns

        temp_df['total_rad_1h:J'] = temp_df['diffuse_rad_1h:J'] + temp_df['direct_rad_1h:J']


        self.keep_columns_exist = [col for col in self.keep_columns if col in temp_df.columns]
        self.interpolate_na(temp_df, self.keep_columns_exist)
        temp_df = temp_df[self.keep_columns_exist]

        # Add hour of day
        temp_df['hour'] = temp_df['ds'].dt.hour
        ml.utils.map_hour_to_seasonal(temp_df, 'hour')
        temp_df = temp_df.drop('ds', axis=1)

        num_features = len(self.keep_columns) - 1

        if(has_target_col):
            temp_df['y'].fillna(df['y'].interpolate().cummax(), inplace=True)

        self.scaler = StandardScaler()
        temp_np = self.scaler.fit_transform(temp_df)
        features = temp_np[:, : num_features]

        pca = PCA(n_components=self.pca_dimensions)
        pca_features = pca.fit_transform(features)

        if(has_target_col):
            target = temp_np[:, num_features]
            pca_w_target = np.column_stack((pca_features, target))

            col_names = [f"PCA{i}" for i in range(self.pca_dimensions)] + ['y']

            temp_df = pd.DataFrame(data=pca_w_target, columns=col_names)
        else:
            col_names = [f"PCA{i}" for i in range(self.pca_dimensions)] + ['y']
            temp_df = pd.DataFrame(data=pca_features, columns=col_names)

        return temp_df

    def train(self, df):
        """
        """
        df = self.preprocess(df)

        PCA_features = df.drop('y', axis=1, inplace=False)

        self.model = LinearRegression()

        self.model.fit(PCA_features, df['y'])


    def predict(self, df):
        df = self.preprocess(df)

        if('y' in df.columns):
            df = df.drop('y', axis=1, inplace=False)

        y_preds = self.model.predict(df)
        y_preds_arr = np.array(y_preds)

        out_df = pd.DataFrame(y_preds_arr, columns=["y_pred"]) 

        # SCALE BACK OUTPUT
        temp_scaler = StandardScaler()
        index_of_y = self.keep_columns.index("y")
        temp_scaler.mean_, temp_scaler.scale_ = self.scaler.mean_[index_of_y], self.scaler.scale_[index_of_y]

        scaled_model_out = temp_scaler.inverse_transform(out_df)
        out_df = pd.DataFrame(data=scaled_model_out, columns=['y_pred'])

        out_df['y_pred'] = out_df['y_pred'].apply(lambda a : max(0, a))

        #print("NUM NEGATIVE OUTPUTS", (out_df['y_pred'] < 0).sum(), "AVERAGE NEG VAL", (out_df[out_df['y_pred'] < 0].mean()))

        return out_df

#### 94. TreeComposite with MetaLearner


In [ ]:
class TreeCompositeMetaLearnerHenrik(MetaModel):
    
    def __init__(self):
        super().__init__("TreeCompositeMetaLearner Henrik")
        self.features = []
        
        self.features.extend(['month',
                             'hour',
                            'total_rad_1h:J',
        'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
       'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
       'dew_point_2m:K', 'effective_cloud_cover:p', 'elevation:m',
       'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
       'fresh_snow_3h:cm', 'fresh_snow_6h:cm',
       'is_in_shadow:idx', 'msl_pressure:hPa', 'precip_5min:mm',
       'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
       'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
       'sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm',
       'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2',
       'sun_azimuth:d', 'sun_elevation:d', 'super_cooled_liquid_water:kgm2',
       't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
       'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
       'wind_speed_w_1000hPa:ms'])
        
        """

        self.features.extend(['month',
                             'hour',
                            'total_rad_1h:J',
                            'fresh_snow_12h:cm',
                            'snow_water:kgm2',
                            'is_day:idx',
                            'is_in_shadow:idx',
                            'rain_water:kgm2',
                            'sun_azimuth:d',
                            'sun_elevation:d',
                            't_1000hPa:K',
                            'dew_or_rime:idx',
                            'air_density_2m:kgm3',
                            'absolute_humidity_2m:gm3'])
        

        self.features.extend(['super_cooled_liquid_water:kgm2',
                              'effective_cloud_cover:p', 'elevation:m',
                              'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
                              'msl_pressure:hPa', 'precip_5min:mm', 'prob_rime:p', 
                              'relative_humidity_1000hPa:p', 'visibility:m'
                              ])
        """                              
        
    def preprocess(self, df: pd.DataFrame):
        """
        """
        temp_df = df.copy()

        has_target = 'y' in temp_df.columns        
        
        ##################################################################################### 
        # FEATURE ENGINEERING
        #####################################################################################

        temp_df['total_rad_1h:J'] = temp_df['diffuse_rad_1h:J'] + temp_df['direct_rad_1h:J']    
        
        # Extracting hour-of-day and month, and making them cyclical
        temp_df['hour'] = temp_df['ds'].dt.hour
        ml.utils.map_hour_to_seasonal(temp_df, 'hour')

        temp_df['month'] = temp_df['ds'].dt.month
        ml.utils.map_month_to_seasonal(temp_df, 'month')
   
        # SETTING NAN TO 0 CONFORMING TO XGBOOST
        temp_df.fillna(0, inplace=True)

        #####################################################################################

        # DROPPING UNEEEDED FEATURES
        if(has_target):
            features_w_y = self.features + ['y']
            temp_df = temp_df[features_w_y]

        else:
            temp_df = temp_df[self.features]

        return temp_df

    def train(self, df: MetaModel):
        """
        """

        self.models = {
            "XGBoost Henrik": XGBoostHenrik(),
            "LightGBM Henrik": LightBGMHenrik()
        }

        for key in self.models:
            self.models[key].train(df)



    def predict(self, df):
        """
        """
        
        all_preds = None

        out_df = None

        for key in self.models:
            y_pred = self.models[key].predict(df)['y_pred']
            if(all_preds is None):
                all_preds = pd.DataFrame(y_pred)
            else:
                all_preds[key] = y_pred.values

            
        avg_series = all_preds.mean(axis=1)

        print("The different models produced the following predictions:")
        print(all_preds)
        print("Averages")
        print(avg_series)

        return pd.DataFrame(avg_series, columns=['y_pred'])